# Music / Lyric Analyzer

## Webscrapping the songs from Wikipedia (1959 - 2021)
#### Notice: There will be a part 2, where i fetch all the lyrics from a site called https://www.mldb.org

OBS:! I saw that most songs have a link. With some aditional data. Genre, Length, Label, Released.
Also all most of singers actually!

In [178]:
import bs4
import requests

def get_top100_song_year(year=2021):
    """This function will return a dictionary with any of the official Billboard American top 100 end of the year lists.
    The top 100 list was introduced in 1959. Before that it was a top 50 only. Returns by default 2021.
    link: https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2021"""
    result_dict = dict()
    
    url = "https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_" + str(year)
    
    r = requests.get(url)
    r.raise_for_status()
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    
    table = soup.select('.wikitable > tbody > tr')
    
    for el in table:
        td = el.select('td')
        if(len(td) > 0):
            number = td[0].text.rstrip('\n')
            title = td[1].text.rstrip('\n')
            artist = extract_artists(td[2])
            title_url = extract_title_url(td[1])
            result_dict[number] = [title[1:-1], artist[0], artist[1], title_url]
    
    return result_dict

def extract_artists(element):
    """This function will return a list of the artists that have made the song. Only 2 artists will be applied,
    if more than two artists have collaborated. 
    """
    list_of_artists = [x.text.rstrip('\n') for x in element]
            
    result_list = split_featuring(list_of_artists)  
    
    # Need to add a blank spot if there is no featuring artist.
    if(len(result_list) < 2):
        result_list.append("")
        
    # Some featurings starts with ' & ' and some ends with 'and'. Quick fix:
    if(' and ' in result_list[1][-5:]):
        result_list[1] = result_list[1][:-5]
    if(' & ' in result_list[1][:3]):
        result_list[1] = result_list[1][3:]
        
    return [artist for artist in result_list][:2]


def split_featuring(list):
        """Many of the songs have featuring artists, sometimes even an additional ' and ' artist.
        This function splits them and returns a new list.
        
        Examples:
        1. Blackstreet featuring Mýa, Mase and Blinky Blink   # featuring + and + ,
        2. Silk Sonic (Bruno Mars and Anderson .Paak)         # Tuples
        3. SpotemGottem featuring Pooh Shiesty or DaBaby      # featuring + or
        4. Gerry and the Pacemakers                           # ' and the ' was a common band name in the old days
        5. Puff Daddy & the Family featuring The Notorious B.I.G.  # & the Family.. Gave some troubles.. 
        """
        
        new_list = []
        
        for artist in list:
            if(' featuring ' in artist):
                [new_list.append(x) for x in artist.split(' featuring ')]
            elif(' and ' in artist and ' and the ' not in artist ):
                [new_list.append(x) for x in artist.split(' and ')]
            elif(' or ' in artist):
                [new_list.append(x) for x in artist.split(' or ')] 
            else:
                new_list.append(artist)
                
        return [x for x in new_list if
                x != ' and ' and x != '' and x != ' (' and x != ')' and 
                x != ', ' and x != ' & 'and x != ' with ' and x != ' & the Family']
            

def extract_title_url(element):
    """This function will return the url for the song. This will be used later on to extract more data about each song."""
    
    if(element.find('a')):
        return element.select('a')[0].get('href')
    else:
        return ""

#get_top100_song_year(1998)

In [179]:
def get_all_top100_song_period(start=1959, end=2021):
    """This function will return..."""
    result_dict = dict()
    for i in range(start, end + 1):
        result_dict[i] = get_top100_song_year(i)
    return result_dict

        
billboard_data = get_all_top100_song_period(1959, 2021)
print("Size of Data:",len(billboard_data))
print("Example (2021):")
billboard_data.get(2021)


Size of Data: 63
Example (2021):


{'1': ['Levitating', 'Dua Lipa', '', '/wiki/Levitating_(song)'],
 '2': ['Save Your Tears',
  'The Weeknd',
  'Ariana Grande',
  '/wiki/Save_Your_Tears'],
 '3': ['Blinding Lights', 'The Weeknd', '', '/wiki/Blinding_Lights'],
 '4': ['Mood', '24kGoldn', 'Iann Dior', '/wiki/Mood_(song)'],
 '5': ['Good 4 U', 'Olivia Rodrigo', '', '/wiki/Good_4_U'],
 '6': ['Kiss Me More', 'Doja Cat', 'SZA', '/wiki/Kiss_Me_More'],
 '7': ['Leave the Door Open',
  'Silk Sonic',
  'Bruno Mars',
  '/wiki/Leave_the_Door_Open'],
 '8': ['Drivers License',
  'Olivia Rodrigo',
  '',
  '/wiki/Drivers_License_(song)'],
 '9': ['Montero (Call Me by Your Name)',
  'Lil Nas X',
  '',
  '/wiki/Montero_(Call_Me_by_Your_Name)'],
 '10': ['Peaches',
  'Justin Bieber',
  'Daniel Caesar',
  '/wiki/Peaches_(Justin_Bieber_song)'],
 '11': ['Butter', 'BTS', '', '/wiki/Butter_(song)'],
 '12': ['Stay',
  'The Kid Laroi',
  'Justin Bieber',
  '/wiki/Stay_(The_Kid_Laroi_and_Justin_Bieber_song)'],
 '13': ['Deja Vu',
  'Olivia Rodrigo',
  '

## Turn it into a dataframe (IMPORTANT)

In [180]:
import pandas as pd

def make_data_into_dataframe(billboard_data):
    data_list = []
    for year, value in billboard_data.items():
        for place, song in value.items():
            list_song = [year, place, song[0], song[1], song[2], song[3]]
            data_list.append(list_song)
    return pd.DataFrame(data_list, columns=['Year', 'Place', 'Title', 'Artist', 'Featuring','Title_url'])
   
make_data_into_dataframe(billboard_data)

,Year,Place,Title,Artist,Featuring,Title_url
0,1959,1,The Battle of New Orleans,Johnny Horton,,/wiki/The_Battle_of_New_Orleans
1,1959,2,Mack the Knife,Bobby Darin,,/wiki/Mack_the_Knife
2,1959,3,Personality,Lloyd Price,,/wiki/Personality_(Lloyd_Price_song)
3,1959,4,Venus,Frankie Avalon,,/wiki/Venus_(Frankie_Avalon_song)
4,1959,5,Lonely Boy,Paul Anka,,/wiki/Lonely_Boy_(Paul_Anka_song)
...,...,...,...,...,...,...
6296,2021,96,Things a Man Oughta Know,Lainey Wilson,,/wiki/Things_a_Man_Oughta_Know
6297,2021,97,Throat Baby (Go Baby),BRS Kash,,/wiki/Throat_Baby_(Go_Baby)
6298,2021,98,Tombstone,Rod Wave,,/wiki/Tombstone_(song)
6299,2021,99,Drinkin' Beer. Talkin' God. Amen.,Chase Rice,Florida Georgia Line,/wiki/Drinkin%27_Beer._Talkin%27_God._Amen.


## Save as CSV. So i dont have to scrape it everyday..

In [181]:
#df_to_csv = pd.DataFrame.from_dict(billboard_data) 
make_data_into_dataframe(billboard_data).to_csv ('data/raw_top100_1959_2021.csv', index = False, header=True)

## Load Data

In [182]:
import pandas as pd
df = pd.read_csv('data/raw_top100_1959_2021.csv' )
df

,Year,Place,Title,Artist,Featuring,Title_url
0,1959,1,The Battle of New Orleans,Johnny Horton,NaN,/wiki/The_Battle_of_New_Orleans
1,1959,2,Mack the Knife,Bobby Darin,NaN,/wiki/Mack_the_Knife
2,1959,3,Personality,Lloyd Price,NaN,/wiki/Personality_(Lloyd_Price_song)
3,1959,4,Venus,Frankie Avalon,NaN,/wiki/Venus_(Frankie_Avalon_song)
4,1959,5,Lonely Boy,Paul Anka,NaN,/wiki/Lonely_Boy_(Paul_Anka_song)
...,...,...,...,...,...,...
6296,2021,96,Things a Man Oughta Know,Lainey Wilson,NaN,/wiki/Things_a_Man_Oughta_Know
6297,2021,97,Throat Baby (Go Baby),BRS Kash,NaN,/wiki/Throat_Baby_(Go_Baby)
6298,2021,98,Tombstone,Rod Wave,NaN,/wiki/Tombstone_(song)
6299,2021,99,Drinkin' Beer. Talkin' God. Amen.,Chase Rice,Florida Georgia Line,/wiki/Drinkin%27_Beer._Talkin%27_God._Amen.


## Testing the data:

In [56]:
df[df['Featuring'] == 'the Pacemakers']

,Year,Place,Title,Artist,Featuring,Title_url
548,1964,49,Don't Let the Sun Catch You Crying,Gerry,the Pacemakers,/wiki/Don%27t_Let_the_Sun_Catch_You_Crying
588,1964,89,How Do You Do It?,Gerry,the Pacemakers,/wiki/How_Do_You_Do_It%3F
642,1965,43,Ferry Cross the Mersey,Gerry,the Pacemakers,/wiki/Ferry_Cross_the_Mersey


In [80]:
df[df['Artist'] == 'Gerry and the Pacemakers'] # Fixed

,Year,Place,Title,Artist,Featuring,Title_url
548,1964,49,Don't Let the Sun Catch You Crying,Gerry and the Pacemakers,NaN,/wiki/Don%27t_Let_the_Sun_Catch_You_Crying
588,1964,89,How Do You Do It?,Gerry and the Pacemakers,NaN,/wiki/How_Do_You_Do_It%3F
642,1965,43,Ferry Cross the Mersey,Gerry and the Pacemakers,NaN,/wiki/Ferry_Cross_the_Mersey


In [57]:
df[df['Featuring'] == ' & ']

,Year,Place,Title,Artist,Featuring,Title_url
36,1959,37,"Kookie, Kookie (Lend Me Your Comb)",Edd Byrnes,&,"/wiki/Kookie,_Kookie_(Lend_Me_Your_Comb)"
126,1960,27,Baby (You've Got What It Takes),Dinah Washington,&,/wiki/Baby_(You%27ve_Got_What_It_Takes)
148,1960,49,Finger Poppin' Time,Hank Ballard,&,/wiki/Finger_Poppin%27_Time
174,1960,75,A Rockin' Good Way (To Mess Around and Fall in...,Dinah Washington,&,/wiki/A_Rockin%27_Good_Way_(To_Mess_Around_and...
316,1962,17,Slow Twistin',Chubby Checker,&,/wiki/Slow_Twistin%27
517,1964,18,Little Children,Billy J. Kramer,&,/wiki/Little_Children_(song)
533,1964,34,Let It Be Me,Betty Everett,&,/wiki/Let_It_Be_Me_(The_Everly_Brothers_song)
550,1964,51,The Girl from Ipanema,Stan Getz,&,/wiki/The_Girl_from_Ipanema
596,1964,97,Bad to Me,Billy J. Kramer,&,/wiki/Bad_to_Me
633,1965,34,Game of Love,Wayne Fontana,&,/wiki/The_Game_of_Love_(Wayne_Fontana_song)


In [86]:
df[df['Featuring'] == ' & '] # Fixed

,Year,Place,Title,Artist,Featuring,Title_url


In [87]:
df[df['Artist'] == 'The Mamas & the Papas']

,Year,Place,Title,Artist,Featuring,Title_url
700,1966,1,California Dreamin',The Mamas & the Papas,NaN,/wiki/California_Dreamin%27
723,1966,24,"Monday, Monday",The Mamas & the Papas,NaN,"/wiki/Monday,_Monday"
861,1967,62,Dedicated to the One I Love,The Mamas & the Papas,NaN,/wiki/Dedicated_to_the_One_I_Love


In [88]:
df[df['Title'] == 'Stay']

,Year,Place,Title,Artist,Featuring,Title_url
197,1960,98,Stay,Maurice Williams and the Zodiacs,NaN,/wiki/Stay_(Maurice_Williams_song)
3345,1992,45,Stay,Shakespears Sister,NaN,/wiki/Stay_(Shakespears_Sister_song)
3580,1994,80,Stay,Eternal,NaN,/wiki/Stay_(Eternal_song)
5413,2013,13,Stay,Rihanna,Mikky Ekko,/wiki/Stay_(Rihanna_song)
5817,2017,17,Stay,Zedd,Alessia Cara,/wiki/Stay_(Zedd_and_Alessia_Cara_song)
6212,2021,12,Stay,The Kid Laroi,Justin Bieber,/wiki/Stay_(The_Kid_Laroi_and_Justin_Bieber_song)


In [90]:
df[df['Title'] == 'Venus']

,Year,Place,Title,Artist,Featuring,Title_url
3,1959,4,Venus,Frankie Avalon,NaN,/wiki/Venus_(Frankie_Avalon_song)
1133,1970,33,Venus,Shocking Blue,NaN,/wiki/Venus_(Shocking_Blue_song)
2738,1986,38,Venus,Bananarama,NaN,/wiki/Venus_(Shocking_Blue_song)


In [91]:
df[df['Artist'] == 'The Beatles']

,Year,Place,Title,Artist,Featuring,Title_url
500,1964,1,I Want to Hold Your Hand,The Beatles,NaN,/wiki/I_Want_to_Hold_Your_Hand
501,1964,2,She Loves You,The Beatles,NaN,/wiki/She_Loves_You
512,1964,13,A Hard Day's Night,The Beatles,NaN,/wiki/A_Hard_Day%27s_Night_(song)
513,1964,14,Love Me Do,The Beatles,NaN,/wiki/Love_Me_Do
515,1964,16,Please Please Me,The Beatles,NaN,/wiki/Please_Please_Me_(song)
539,1964,40,Twist and Shout,The Beatles,NaN,/wiki/Twist_and_Shout
551,1964,52,Can't Buy Me Love,The Beatles,NaN,/wiki/Can%27t_Buy_Me_Love
554,1964,55,Do You Want to Know a Secret,The Beatles,NaN,/wiki/Do_You_Want_to_Know_a_Secret
594,1964,95,I Saw Her Standing There,The Beatles,NaN,/wiki/I_Saw_Her_Standing_There
606,1965,7,Help!,The Beatles,NaN,/wiki/Help!_(song)


#### Testing featuring after refactoring the web scrapper:

In [106]:
# How many songs have a featurin artist for the song:
print("Featuring artists: ",df['Artist'].str.contains('featuring').sum())
print("AND in the name: ",df['Artist'].str.contains('and').sum()) # OBS.: This could also be Andrew, Andy etc.. not valid!

Featuring artists:  0
AND in the name:  227


In [183]:
print("Featuring artists: ",df['Featuring'].str.contains('&').sum())

Featuring artists:  4


In [167]:
test_featuring_for_and[test_featuring_for_and['Featuring'].str.contains(' and ')]

,Year,Place,Title,Artist,Featuring,Title_url
6149,2020,49,For the Night,Pop Smoke,Lil Baby and DaBaby,/wiki/For_the_Night
6221,2021,21,34+35,Ariana Grande,Doja Cat and,/wiki/34%2B35
6259,2021,59,Every Chance I Get,DJ Khaled,Lil Baby and Lil Durk,/wiki/Every_Chance_I_Get_(song)
6260,2021,60,Essence,Wizkid,Justin Bieber and,/wiki/Essence_(song)


In [185]:
# It's usually used if there is a third featuring person/band. DRAKE featuring 21 SAVAGE and PROJECT PAT
# Might have to change the way i crawl the data!

test_featuring_for_and = df.dropna(subset='Featuring')

test_featuring_for_and[test_featuring_for_and['Featuring'].str.contains(' and')]
#df[df['Featuring'].str.contains(' and')]

,Year,Place,Title,Artist,Featuring,Title_url
6149,2020,49,For the Night,Pop Smoke,Lil Baby and DaBaby,/wiki/For_the_Night
6259,2021,59,Every Chance I Get,DJ Khaled,Lil Baby and Lil Durk,/wiki/Every_Chance_I_Get_(song)


In [186]:
#testdf['Year'].values.tolist()

for x in test_featuring_for_and.values:
    print(x[0], x[1], x[4])

1959 37 Connie Stevens
1959 60 Bob
1959 81 Carol Kay
1959 95 Dean
1960 27 Brook Benton
1960 49 The Midnighters
1960 62 The Phantom's Band
1960 75 Brook Benton
1960 79 Flip
1961 27 Dee Dee
1961 65 Tina Turner
1962 17 Dee Dee Sharp
1962 50 Juan
1962 84 Dee Dee
1962 98 Mary
1963 7 Paula
1963 16 Mary
1963 17 Mary
1963 18 Grace
1963 25 Charlie Foxx
1963 28 Dean
1963 53 the Enchanters
1964 18 The Dakotas
1964 28 Dean
1964 30 Gordon
1964 34 Jerry Butler
1964 48 Dean
1964 51 Astrud Gilberto
1964 97 The Dakotas
1965 34 The Mindbenders
1965 75 Gordon
1965 82 Gordon
1966 79 Gordon
1967 7 Nancy Sinatra
1967 32 Tammi Terrell
1967 87 Tammi Terrell
1967 91 Mary
1968 57 Tammi Terrell
1968 82 Tammi Terrell
1969 25 Billy Preston
1969 78 The First Edition
1969 87 The Temptations
1970 20 War
1970 38 The First Edition
1971 10 Dawn
1971 22 Linda McCartney
1971 67 Friends
1971 71 Katie Kissoon
1972 38 Tim
1972 58 Donny Hathaway
1972 69 His Lost Planet Airmen
1972 84 Brenda Lee Eager
1973 1 Dawn
1973 34 Dawn


In [102]:
df[df['Artist'] == 'Dionne'] # Not perfect.. This should have been Dionne Warwick and a bunch of featuring artists

,Year,Place,Title,Artist,Featuring,Title_url
2701,1986,1,That's What Friends Are For,Dionne,Friends (,/wiki/That%27s_What_Friends_Are_For


In [96]:
df[df['Title'] == 'Leave the Door Open'] # This title's artist got '()' in the name....FIXED

,Year,Place,Title,Artist,Featuring,Title_url
6207,2021,7,Leave the Door Open,Silk Sonic,Bruno Mars,/wiki/Leave_the_Door_Open


In [187]:
df[df['Artist'] == 'Puff Daddy'] # & the Family is not valid...

,Year,Place,Title,Artist,Featuring,Title_url
3803,1997,3,I'll Be Missing You,Puff Daddy,Faith Evans,/wiki/I%27ll_Be_Missing_You
3805,1997,5,Can't Nobody Hold Me Down,Puff Daddy,Mase,/wiki/Can%27t_Nobody_Hold_Me_Down
3919,1998,19,Been Around the World,Puff Daddy,The Notorious B.I.G.,/wiki/Been_Around_the_World
3947,1998,47,Come with Me,Puff Daddy,Jimmy Page,/wiki/Come_with_Me_(Puff_Daddy_song)
3968,1998,68,Victory,Puff Daddy,The Notorious B.I.G.,/wiki/Victory_(Puff_Daddy_song)
4095,1999,95,Satisfy You,Puff Daddy,R. Kelly,/wiki/Satisfy_You_(Puff_Daddy_song)


In [92]:
df[df['Title'] == 'Peaches'] # This title's artist(s) ot featuring + and in the name. FIXED

,Year,Place,Title,Artist,Featuring,Title_url
6210,2021,10,Peaches,Justin Bieber,Daniel Caesar,/wiki/Peaches_(Justin_Bieber_song)


In [97]:
print("Number of songs: ",len(df))
print("Years: ", len(df["Year"].drop_duplicates()))

Number of songs:  6301
Years:  63


In [98]:
# Find all the no.1 songs

df[df['Place'] == "1"]

,Year,Place,Title,Artist,Featuring,Title_url
0,1959,1,The Battle of New Orleans,Johnny Horton,NaN,/wiki/The_Battle_of_New_Orleans
100,1960,1,Theme from A Summer Place,Percy Faith,NaN,/wiki/Theme_from_A_Summer_Place
200,1961,1,Tossin' and Turnin',Bobby Lewis,NaN,/wiki/Tossin%27_and_Turnin%27
300,1962,1,Stranger on the Shore,Acker Bilk,NaN,/wiki/Stranger_on_the_Shore
400,1963,1,Sugar Shack,Jimmy Gilmer and the Fireballs,NaN,/wiki/Sugar_Shack
...,...,...,...,...,...,...
5801,2017,1,Shape of You,Ed Sheeran,NaN,/wiki/Shape_of_You
5901,2018,1,God's Plan,Drake,NaN,/wiki/God%27s_Plan_(song)
6001,2019,1,Old Town Road,Lil Nas X,Billy Ray Cyrus,/wiki/Old_Town_Road
6101,2020,1,Blinding Lights,The Weeknd,NaN,/wiki/Blinding_Lights


In [99]:
songs_without_url = df[df['Title_url'].isnull()]

In [100]:
# Find songs that don't have a link.
            
print(len(songs_without_url), " out of ", len(df), " Songs.",  (len(songs_without_url) / len(df))*100, "%")
songs_without_url

74  out of  6301  Songs. 1.1744167592445642 %


,Year,Place,Title,Artist,Featuring,Title_url
112,1960,13,Greenfields,The Brothers Four,NaN,NaN
139,1960,40,Way Down Yonder in New Orleans,Freddy Cannon,NaN,NaN
165,1960,66,It's Time to Cry,Paul Anka,NaN,NaN
181,1960,82,Lady Luck,Lloyd Price,NaN,NaN
182,1960,83,Step by Step,The Crests,NaN,NaN
...,...,...,...,...,...,...
4061,1999,61,If You,Silk,NaN,NaN
4081,1999,81,Faded Pictures,Case,Joe,NaN
4289,2001,89,I'm a Thug,Trick Daddy,NaN,NaN
4388,2002,88,Anything,Jaheim,Next,NaN


### Notes after testing

In [115]:
# Notice 1986: Dionne and Friends (Dionne Warwick, Gladys Knight, Elton John and Stevie Wonder)
# Need a a clever way to break that down.
# Also 2004, 2012 (and others) have "feature" or "and". Need to be able to break down aswell 
# 74 songs wont get extra data. Which is acceptable
# Paul McCartney need some attention. "Paul", "Paul M and the Wings", "Beatles"


## Get additional data from wikipedia link

In [113]:
import bs4
import requests
import re


def get_additional_song_data(link):
    result_dict = dict()
    
    url = "https://en.wikipedia.org/"+link
    
    r = requests.get(url)
    r.raise_for_status()
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    
    # Some songs have more than one infobox. We only want the first.
    infobox = soup.select('.infobox')
    table = infobox[0].select('tbody > tr')
    
    # All the data we want to extract (Will convert it into an Object!)
    released = ""
    genres = [""]
    length = ""
    label = ""
    total_labels = ""
    writer = [""]
    youtube = ""
    
    
    for el in table:

        
        if(el.find('th')):
            th = el.select('th')
            td = el.select('td')
            if(len(td) > 0):
                #print(th[0].text, td[0].text)
                match th[0].text:
                    case "Released":
                        released =  extract_released_year(td[0])
                        #print("Released", extract_released_year(td[0]))
                    case "Genre":
                        genres = extract_genre(td[0])
                        #print("Genre:", extract_genre(td[0]))
                    case "Length":
                        length = extract_length(td[0])
                        #print("Length", extract_length(td[0]))
                    case "Label":
                        label_data = extract_label(td[0])
                        label = label_data[0]
                        total_labels = label_data[1]
                        #print("Label", extract_label(td[0]))
                    case "Songwriter(s)":
                        writer = extract_songwriter(td[0])
                        #print("Songwriter(s)", extract_songwriter(td[0]))
                        
        # Search for YT link
        if(el.find('a', {'title': 'YouTube'})):
            youtube = el.find('a', {'class': 'external'}).get('href')
            #print(el.find('a', {'class': 'external'}).get('href'))
                    
                
    return [released, genres, length, label, total_labels, writer, youtube]
    
    

def extract_released_year(element):
    """This function will return the year the song was released. But tests shows that is different scenarios. 
    1. Multiple release days, release etc., 
    2. Sometimes only the Year and Month(or Just the year), 
    
    I am using regex and split to make a list of 4 digit numbers and only returns index 0(Incase there was a rerelease later)
    """    
    years_reg = re.compile(r"\b(19|20)\d{2}\b")
        
    return [year for year in element.text.split() if re.search(years_reg, year)][0]
    
                        
def extract_genre(element):
    """This function will return a list of up to 3 genres. But tests shows that is different scenarios. 
    1. Only one Genre, 
    2. Multiple genres in an unordered list(ul), 
    3. Multiple genres with a bookmark like this. Pop[1], Rock[2].
    
    Which is handle but simple if statements.
    """
    letters_reg = re.compile(r"[a-zA-Z]")    
    
    if(element.find('ul')):
        result_list = [x.text for x in element.select('a')]
    else:
        result_list = element.text.split()
        
    return [genre for genre in result_list if re.search(letters_reg, genre)][:3]
    
    
def extract_length(element):
    """This function will return the length of the song. In some cases there is a single and an album version etc.,
    but i decided to return just the longest version. 
    1. The function splits the text elements into a list if it contains ':'.
    2. It cleans up the result for any letters or symbols(not ':' obviously).
    3. Return the max value."""

    # c = character   l = length
    lengths = ["".join([c for c in l if int(c.isnumeric()) or c == ":"]) for l in element.text.split() if ":" in l] 
    
    return max(lengths)
    
def extract_label(element):
    """This function will return only the first mentioned Label. Some singers/song change labels for whatever reason.
    Sometimes up tp two or three times. For simplicity I only return the first mentioned and the amount of different
    labels that the song have had. Maybe we can use that information for something interresting."""
    
    if(element.find('a')):
        result_list = [x.text for x in element.select('a')]
    elif(element.find('li')):
        result_list = [x.text for x in element.select('li')]
    else:
        result_list = element.text.split()
    
    # Returning originale label and the amount of different labels.
    return [[label for label in result_list][0], len(result_list)]

def extract_songwriter(element):
    """This function will return only the first mentioned songwriter. Some singers/song change labels for whatever reason.
    Sometimes up tp two or three times. For simplicity I only return the first mentioned and the amount of different
    labels that the song have had. Maybe we can use that information for something interresting."""
    
    if(element.find('a')):
        result_list = [x.text for x in element.select('a')]
        
    else:
        result_list = re.split('; |,',element.text) 
    
    # Returning originale label and the amount of different labels.
    return [writer for writer in result_list][:2]
    
    
    
get_additional_song_data("/wiki/Thrift_Shop")

# Some songs have up to 7 genres! Limit it to 3!?
# Length can be different aswell, album and single (Think i should go with the longest one)
# Multiple recording days. (Take the earliest)
# More than one label! (Like Wham - Careless whispers...)
# Song writers is often more than one. Some weird ones like Andy Gibb - shadow song:  "Barry, Robin & Maurice Gibb; Andy Gibb"

['2012',
 ['Pop-rap', 'comedy hip hop'],
 '3:55',
 'Macklemore LLC',
 1,
 ['Ben Haggerty', 'Ryan Lewis'],
 'https://www.youtube.com/watch?v=QK8mJJJvaes']

In [106]:
test1 = "/wiki/The_First_Time_Ever_I_Saw_Your_Face" # 2 different lenghts + label also have a number after
test2 = "/wiki/When_Doves_Cry" # 7 different genres! And 3  dates..
test3 = "/wiki/Shadow_Dancing_(song)" # Songwriters: "Barry, Robin & Maurice Gibb; Andy Gibb"
test4 = "/wiki/Careless_Whisper" # Genre has this [1] etc + multiple labels
test5 = "/wiki/The_Way_We_Were_(song)" # No genre
test6 = "/wiki/The_Sign_(song)" # the genres prints like this: Techno-reggaepopEuropop (Make sure it's splits correctly)

get_additional_song_data(test1)
get_additional_song_data(test2)
get_additional_song_data(test3)
get_additional_song_data(test4)
get_additional_song_data(test5)
get_additional_song_data(test6)

Released 1972
Genre: ['Soul', 'vocal jazz']
Length 5:22
Label ['Atlantic', 2]
Songwriter(s) ['Ewan MacColl']
Released 1984
Genre: ['Experimental pop', 'neo-psychedelia', 'soul']
Length 5:52
Label ['Warner', 2]
Songwriter(s) ['Prince']
https://www.youtube.com/watch?v=UG3VcCAlUgE
Released 1978
Genre: ['Disco']
Length 4:34
Label ['RSO', 2]
Songwriter(s) ['Barry, Robin & Maurice Gibb', 'Andy Gibb']
Released 1984
Genre: ['Pop', 'soul', 'R&B']
Length 6:30
Label ['Epic', 3]
Songwriter(s) ['George Michael', 'Andrew Ridgeley']
https://www.youtube.com/watch?v=izGwDsrQ1eQ
Released 1973
Length 3:29
Label ['Columbia', 1]
Songwriter(s) ['Alan Bergman', 'Marilyn Bergman']
Released 1993
Genre: ['Techno-reggae', 'pop', 'Europop']
Length 3:08
Label ['Arista', 2]
Songwriter(s) ['Jonas Berggren']
https://www.youtube.com/watch?v=iqu132vTl5Y


In [13]:
list123 = [['"The Battle of New Orleans"', 'Johnny Horton', '/wiki/The_Battle_of_New_Orleans']
,['"Theme from A Summer Place"', 'Percy Faith', '/wiki/Theme_from_A_Summer_Place']
,['"Tossin\' and Turnin\'"', 'Bobby Lewis', '/wiki/Tossin%27_and_Turnin%27']
,['"Stranger on the Shore"', 'Acker Bilk', '/wiki/Stranger_on_the_Shore']
,['"Sugar Shack"', 'Jimmy Gilmer and the Fireballs', '/wiki/Sugar_Shack']
,['"I Want to Hold Your Hand"', 'The Beatles', '/wiki/I_Want_to_Hold_Your_Hand']
,['"Wooly Bully"', 'Sam the Sham and the Pharaohs', '/wiki/Wooly_Bully']
,['"California Dreamin\'"', 'The Mamas & the Papas', '/wiki/California_Dreamin%27']
,['"To Sir With Love"', 'Lulu', '/wiki/To_Sir_with_Love_(song)']
,['"Hey Jude"', 'The Beatles', '/wiki/Hey_Jude']
,['"Sugar, Sugar"', 'The Archies', '/wiki/Sugar,_Sugar']
,['"Bridge Over Troubled Water"', 'Simon & Garfunkel', '/wiki/Bridge_Over_Troubled_Water_(song)']
,['"Joy to the World"', 'Three Dog Night', '/wiki/Joy_to_the_World_(Hoyt_Axton_song)']
,['"The First Time Ever I Saw Your Face"', 'Roberta Flack', '/wiki/The_First_Time_Ever_I_Saw_Your_Face']
,['"Tie a Yellow Ribbon Round the Ole Oak Tree"', 'Tony Orlando and Dawn', '/wiki/Tie_a_Yellow_Ribbon_Round_the_Ole_Oak_Tree']
,['"The Way We Were"', 'Barbra Streisand', '/wiki/The_Way_We_Were_(song)']
,['"Love Will Keep Us Together"', 'Captain & Tennille', '/wiki/Love_Will_Keep_Us_Together#Captain_&_Tennille_version']
,['"Silly Love Songs"', 'Wings', '/wiki/Silly_Love_Songs']
,['"Tonight\'s the Night (Gonna Be Alright)"', 'Rod Stewart', '/wiki/Tonight%27s_the_Night_(Gonna_Be_Alright)']
,['"Shadow Dancing"', 'Andy Gibb', '/wiki/Shadow_Dancing_(song)']
,['"My Sharona"', 'The Knack', '/wiki/My_Sharona']
,['"Call Me"', 'Blondie', '/wiki/Call_Me_(Blondie_song)']
,['"Bette Davis Eyes"', 'Kim Carnes', '/wiki/Bette_Davis_Eyes']
,['"Physical"', 'Olivia Newton-John', '/wiki/Physical_(Olivia_Newton-John_song)']
,['"Every Breath You Take"', 'The Police', '/wiki/Every_Breath_You_Take']
,['"When Doves Cry"', 'Prince', '/wiki/When_Doves_Cry']
,['"Careless Whisper"', 'Wham!', '/wiki/Careless_Whisper']
,['"That\'s What Friends Are For"', 'Dionne and Friends (Dionne Warwick, Gladys Knight, Elton John and Stevie Wonder)', '/wiki/That%27s_What_Friends_Are_For']
,['"Walk Like An Egyptian"', 'The Bangles', '/wiki/Walk_Like_An_Egyptian']
,['"Faith"', 'George Michael', '/wiki/Faith_(George_Michael_song)']
,['"Look Away"', 'Chicago', '/wiki/Look_Away']
,['"Hold On"', 'Wilson Phillips', '/wiki/Hold_On_(Wilson_Phillips_song)']
,['"(Everything I Do) I Do It for You"', 'Bryan Adams', '/wiki/(Everything_I_Do)_I_Do_It_for_You']
,['"End of the Road"', 'Boyz II Men', '/wiki/End_of_the_Road_(Boyz_II_Men_song)']
,['"I Will Always Love You"', 'Whitney Houston', '/wiki/I_Will_Always_Love_You#Whitney_Houston_version']
,['"The Sign"', 'Ace of Base', '/wiki/The_Sign_(song)']
,['"Gangsta\'s Paradise"', 'Coolio featuring L.V.', '/wiki/Gangsta%27s_Paradise']
,['"Macarena (Bayside Boys Mix)"', 'Los del Río', '/wiki/Macarena_(song)']
,['"Candle in the Wind 1997" / "Something About the Way You Look Tonight"', 'Elton John', '/wiki/Candle_in_the_Wind_1997']
,['"Too Close"', 'Next', '/wiki/Too_Close_(Next_song)']
,['"Believe"', 'Cher', '/wiki/Believe_(Cher_song)']
,['"Breathe"', 'Faith Hill', '/wiki/Breathe_(Faith_Hill_song)']
,['"Hanging by a Moment"', 'Lifehouse', '/wiki/Hanging_by_a_Moment']
,['"How You Remind Me"', 'Nickelback', '/wiki/How_You_Remind_Me']
,['"In da Club"', '50 Cent', '/wiki/In_da_Club']
,['"Yeah!"', 'Usher featuring Lil Jon and Ludacris', '/wiki/Yeah!_(Usher_song)']
,['"We Belong Together"', 'Mariah Carey', '/wiki/We_Belong_Together_(Mariah_Carey_song)']
,['"Bad Day"', 'Daniel Powter', '/wiki/Bad_Day_(Daniel_Powter_song)']
,['"Irreplaceable"', 'Beyoncé', '/wiki/Irreplaceable']
,['"Low"', 'Flo Rida featuring T-Pain', '/wiki/Low_(Flo_Rida_song)']
,['"Boom Boom Pow"', 'The Black Eyed Peas', '/wiki/Boom_Boom_Pow']
,['"Tik Tok"', 'Kesha', '/wiki/Tik_Tok_(song)']
,['"Rolling in the Deep"', 'Adele', '/wiki/Rolling_in_the_Deep']
,['"Somebody That I Used to Know"', 'Gotye featuring Kimbra', '/wiki/Somebody_That_I_Used_to_Know']
,['"Thrift Shop"', 'Macklemore & Ryan Lewis featuring Wanz', '/wiki/Thrift_Shop']
,['"Happy"', 'Pharrell Williams', '/wiki/Happy_(Pharrell_Williams_song)']
,['"Uptown Funk"', 'Mark Ronson featuring Bruno Mars', '/wiki/Uptown_Funk']
,['"Love Yourself"', 'Justin Bieber', '/wiki/Love_Yourself']
,['"Shape of You"', 'Ed Sheeran', '/wiki/Shape_of_You']
,['"God\'s Plan"', 'Drake', '/wiki/God%27s_Plan_(song)']
,['"Old Town Road"', 'Lil Nas X featuring Billy Ray Cyrus', '/wiki/Old_Town_Road']
,['"Blinding Lights"', 'The Weeknd', '/wiki/Blinding_Lights']
,['"Levitating"', 'Dua Lipa', '/wiki/Levitating_(song)']]

In [114]:
for song in list123:
    print("================")
    print(song[1],song[0])
    print(get_additional_song_data(song[2]))

Johnny Horton "The Battle of New Orleans"
['1959', ['Country'], '2:33', 'Columbia', 1, ['Jimmy Driftwood'], '']
Percy Faith "Theme from A Summer Place"
['1959[4]', ['Easy', 'listening'], '2:25', 'Columbia Records', 1, ['Max Steiner'], 'https://www.youtube.com/watch?v=fRV0yHiEua8']
Bobby Lewis "Tossin' and Turnin'"
['1961', ['R&B'], '2:29', 'Beltone', 1, ['Ritchie Adams'], '']
Acker Bilk "Stranger on the Shore"
['1961', ['Easy', 'listening', 'Jazz'], '2:52', 'Columbia', 2, ['Acker Bilk', 'Robert Mellin'], '']
Jimmy Gilmer and the Fireballs "Sugar Shack"
['1963', ['Garage', 'rock,', 'rock'], '2:00', 'Dot', 1, ['Keith McCormack'], '']
The Beatles "I Want to Hold Your Hand"
['1963', ['Rock and roll', 'pop'], '2:24', 'Parlophone', 2, ['Lennon–McCartney'], 'https://www.youtube.com/watch?v=XT4pwRi2JmY']
Sam the Sham and the Pharaohs "Wooly Bully"
['1965', ['Rock and roll', 'garage rock'], '2:20', 'MGM', 1, ['Domingo Samudio'], '']
The Mamas & the Papas "California Dreamin'"
['1965', ['Sunshin

['2009', [''], '5:08', 'Interscope', 1, ['William Adams', 'Allan Pineda'], 'https://www.youtube.com/watch?v=4m48GqaOz90']
Kesha "Tik Tok"
['2009', ['Dance-pop', 'electropop'], '3:20', 'RCA', 1, ['Kesha Sebert', 'Dr. Luke'], 'https://www.youtube.com/watch?v=iP6XpLQM2Cs']
Adele "Rolling in the Deep"
['2010', ['Rhythm and blues', 'soul'], '3:48', 'XL', 2, ['Adele Adkins', 'Paul Epworth'], 'https://www.youtube.com/watch?v=rYEDA3JcQqw']
Gotye featuring Kimbra "Somebody That I Used to Know"
['2011', ['Art pop', 'alternative rock'], '4:04', 'Eleven', 1, ['Wally de Backer'], '']
Macklemore & Ryan Lewis featuring Wanz "Thrift Shop"
['2012', ['Pop-rap', 'comedy hip hop'], '3:55', 'Macklemore LLC', 1, ['Ben Haggerty', 'Ryan Lewis'], 'https://www.youtube.com/watch?v=QK8mJJJvaes']
Pharrell Williams "Happy"
['2013', ['Soul', 'neo soul'], '3:55', 'i Am Other', 2, ['Pharrell Williams'], 'https://www.youtube.com/watch?v=y6Sxv-sUYtM']
Mark Ronson featuring Bruno Mars "Uptown Funk"
['2014', ['Funk', 'pop

## Web scrape all the lyrics

## Textblob (For the lyrics)
- https://www.youtube.com/watch?v=ea4IadDRwuc&list=PL_92WMXSLe_-RkWW5zAQZ-gMdVqZ7T-_F&index=3
- https://textblob.readthedocs.io/en/dev/quickstart.html

In [150]:
# Research more